# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). 

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**

In [1]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pythresh is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.knn import KNN
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [4]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 
              'KARCH', 'OCSVM', 'CLUST', 'DECOMP']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        print('{thres_name} ROC:{roc}, precision @ rank n:{prn}, '
              'execution time: {duration}s'.format(
        thres_name=thres_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...
Inter-Quartile Region (IQR) ROC:0.838, precision @ rank n:0.5, execution time: 0.7258s
Median Absolute Deviation (MAD) ROC:0.8445, precision @ rank n:0.5, execution time: 0.7998s
Full Width at Full Minimum (FWFM) ROC:0.8522, precision @ rank n:0.5, execution time: 0.8054s
Yeo-Johnson Transformation (YJ) ROC:0.8331, precision @ rank n:0.5357, execution time: 1.115s
Z Score (ZSCORE) ROC:0.8336, precision @ rank n:0.4643, execution time: 0.7207s
AUC Percentage (AUCP) ROC:0.8375, precision @ rank n:0.5, execution time: 0.7186s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:0.8193, precision @ rank n:0.5357, execution time: 0.6943s
Fixed Gradient Descent (FGD) ROC:0.828, precision @ rank n:0.5, execution time: 0.7007s
Distance Shift from Normal (DSN) ROC:0.8499, precision @ rank n:0.5357, execution time: 0.7095s
Trained Classifier (CLF) ROC:0.8431, precision @ rank n:0.5714, execution time: 0.6895s
Filtering Based (FILTER) ROC:0.8424, precision @ rank n:0.6071

Monte Carlo Statistical Tests (MCST) ROC:0.7803, precision @ rank n:0.5217, execution time: 0.5715s
Histogram Based Methods (HIST) ROC:0.7684, precision @ rank n:0.5435, execution time: 0.5352s
Mollifier (MOLL) ROC:0.7757, precision @ rank n:0.5217, execution time: 0.5352s
Chauvenet's Criterion (CHAU) ROC:0.768, precision @ rank n:0.5217, execution time: 0.5321s
Generalized Extreme Studentized Deviate (GESD) ROC:0.7817, precision @ rank n:0.587, execution time: 0.6049s
Modified Thompson Tau Test (MTT) ROC:0.768, precision @ rank n:0.5, execution time: 0.5336s
Karcher Mean (KARCH) ROC:0.7799, precision @ rank n:0.5217, execution time: 0.5362s
One-Class SVM (OCSVM) ROC:0.7757, precision @ rank n:0.5217, execution time: 0.7506s
Clustering (CLUST) ROC:0.7888, precision @ rank n:0.5435, execution time: 0.5799s
Decomposition (DECOMP) ROC:0.7723, precision @ rank n:0.5435, execution time: 0.5899s

... Processing letter.mat ...
Inter-Quartile Region (IQR) ROC:0.6146, precision @ rank n:0.0488,

AUC Percentage (AUCP) ROC:0.9994, precision @ rank n:0.9512, execution time: 1.9618s
Quasi-Monte Carlo Discreperancy (QMCD) ROC:1.0, precision @ rank n:1.0, execution time: 1.5432s
Fixed Gradient Descent (FGD) ROC:0.9999, precision @ rank n:0.9756, execution time: 1.7805s
Distance Shift from Normal (DSN) ROC:1.0, precision @ rank n:1.0, execution time: 1.9834s
Trained Classifier (CLF) ROC:1.0, precision @ rank n:1.0, execution time: 1.449s
Filtering Based (FILTER) ROC:0.9967, precision @ rank n:0.8, execution time: 1.4488s
Topological Winding Number (WIND) ROC:1.0, precision @ rank n:1.0, execution time: 1.9857s
Elliptical Boundary (EB) ROC:0.9994, precision @ rank n:0.9024, execution time: 2.0613s
Regression Intercept (REGR) ROC:1.0, precision @ rank n:1.0, execution time: 1.8763s
Bootstrap Method (BOOT) ROC:1.0, precision @ rank n:1.0, execution time: 2.1531s
Monte Carlo Statistical Tests (MCST) ROC:0.9985, precision @ rank n:0.9024, execution time: 2.184s
Histogram Based Methods (HI

Modified Thompson Tau Test (MTT) ROC:0.6939, precision @ rank n:0.5752, execution time: 0.6453s
Karcher Mean (KARCH) ROC:0.6996, precision @ rank n:0.5664, execution time: 0.5745s
One-Class SVM (OCSVM) ROC:0.6931, precision @ rank n:0.5575, execution time: 0.8012s
Clustering (CLUST) ROC:0.7172, precision @ rank n:0.6018, execution time: 0.6306s
Decomposition (DECOMP) ROC:0.7065, precision @ rank n:0.5664, execution time: 0.7054s

... Processing satellite.mat ...
Inter-Quartile Region (IQR) ROC:0.6986, precision @ rank n:0.5382, execution time: 1.3114s
Median Absolute Deviation (MAD) ROC:0.6967, precision @ rank n:0.5603, execution time: 1.2761s
Full Width at Full Minimum (FWFM) ROC:0.7082, precision @ rank n:0.5542, execution time: 2.7471s
Yeo-Johnson Transformation (YJ) ROC:0.7092, precision @ rank n:0.564, execution time: 3.91s
Z Score (ZSCORE) ROC:0.6643, precision @ rank n:0.5382, execution time: 1.2791s
AUC Percentage (AUCP) ROC:0.7047, precision @ rank n:0.58, execution time: 2.7

Trained Classifier (CLF) ROC:0.8193, precision @ rank n:0.2632, execution time: 0.6294s
Filtering Based (FILTER) ROC:0.8331, precision @ rank n:0.2632, execution time: 0.6289s
Topological Winding Number (WIND) ROC:0.8097, precision @ rank n:0.2105, execution time: 0.7638s
Elliptical Boundary (EB) ROC:0.789, precision @ rank n:0.3158, execution time: 1.1207s
Regression Intercept (REGR) ROC:0.8417, precision @ rank n:0.2632, execution time: 0.7823s
Bootstrap Method (BOOT) ROC:0.8069, precision @ rank n:0.2105, execution time: 1.0025s
Monte Carlo Statistical Tests (MCST) ROC:0.8245, precision @ rank n:0.2632, execution time: 0.9227s
Histogram Based Methods (HIST) ROC:0.7605, precision @ rank n:0.1579, execution time: 0.6588s
Mollifier (MOLL) ROC:0.8131, precision @ rank n:0.2105, execution time: 0.6328s
Chauvenet's Criterion (CHAU) ROC:0.8195, precision @ rank n:0.3158, execution time: 0.6522s
Generalized Extreme Studentized Deviate (GESD) ROC:0.8051, precision @ rank n:0.3158, execution 

In [5]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP
0,arrhythmia,452,274,14.6018,0.7258,0.7998,0.8054,1.115,0.7207,0.7186,...,0.7374,0.6992,0.7882,0.7821,0.8609,0.7671,0.6932,0.8844,0.7049,0.7528
0,cardio,1831,21,9.6122,0.7562,0.6795,0.7862,1.3251,0.715,0.8723,...,1.0138,0.737,0.687,0.6797,1.0592,0.748,0.7283,1.0741,0.8925,1.0052
0,glass,214,9,4.2056,0.5394,0.514,0.5019,0.767,0.5119,0.5441,...,0.5188,0.4999,0.501,0.502,0.5942,0.5565,0.5876,0.6735,0.5016,0.5544
0,ionosphere,351,33,35.8974,0.5245,0.5548,0.5559,0.8756,0.5352,0.6035,...,0.5715,0.5352,0.5352,0.5321,0.6049,0.5336,0.5362,0.7506,0.5799,0.5899
0,letter,1600,32,6.25,0.7946,0.7639,0.8784,1.4111,0.7725,1.0051,...,0.9535,0.6847,0.755,0.7438,0.9909,0.6943,0.7208,1.09,0.855,1.005
0,lympho,148,18,4.0541,0.4935,0.494,0.4954,0.7565,0.4947,0.507,...,0.5304,0.5442,0.5114,0.4996,0.5212,0.4944,0.5,0.7318,0.5311,0.5093
0,mnist,7603,100,9.2069,2.0508,2.0539,4.1545,5.5779,2.0593,4.1089,...,5.6395,2.0803,2.1826,2.0422,3.6604,2.0981,2.9214,5.2154,6.7684,5.6937
0,musk,3062,166,3.1679,1.4525,1.5274,1.787,2.796,1.4316,1.9618,...,2.184,1.4455,1.4936,1.4586,2.0798,1.4488,1.641,2.3972,1.7219,2.1782
0,optdigits,5216,64,2.8758,1.4076,1.4144,2.4132,3.1933,1.3902,2.5823,...,3.1657,1.4694,1.4161,1.4045,2.4387,1.5109,1.8262,3.0653,3.096,3.207
0,pendigits,6870,16,2.2707,1.1929,1.1968,2.8444,4.0194,1.1946,2.8526,...,4.1978,1.173,1.2246,1.2196,2.6155,1.303,2.0354,3.8048,6.1686,4.2511


In [6]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP
0,arrhythmia,452,274,14.6018,0.838,0.8445,0.8522,0.8331,0.8336,0.8375,...,0.8466,0.8492,0.8375,0.8548,0.8576,0.8147,0.8403,0.8172,0.8466,0.8345
0,cardio,1831,21,9.6122,0.9042,0.9166,0.9337,0.9274,0.9298,0.9204,...,0.9207,0.9236,0.9038,0.942,0.9304,0.9261,0.9034,0.9179,0.9274,0.901
0,glass,214,9,4.2056,0.684,0.6642,0.6198,0.6247,0.6494,0.6247,...,0.6716,0.642,0.6469,0.7037,0.6494,0.6346,0.6519,0.642,0.6296,0.6198
0,ionosphere,351,33,35.8974,0.7854,0.7751,0.7698,0.7913,0.7998,0.7778,...,0.7803,0.7684,0.7757,0.768,0.7817,0.768,0.7799,0.7757,0.7888,0.7723
0,letter,1600,32,6.25,0.6146,0.6214,0.6088,0.6052,0.6091,0.6595,...,0.6168,0.5988,0.5677,0.6585,0.5961,0.6206,0.594,0.6753,0.627,0.6129
0,lympho,148,18,4.0541,0.9942,1.0,0.9942,1.0,1.0,1.0,...,1.0,1.0,1.0,0.9942,1.0,1.0,0.9942,0.9942,0.9942,1.0
0,mnist,7603,100,9.2069,0.7663,0.8017,0.7906,0.779,0.8109,0.7757,...,0.7835,0.8053,0.8196,0.7826,0.8127,0.7941,0.7692,0.7976,0.8062,0.7882
0,musk,3062,166,3.1679,1.0,0.9994,0.9999,0.9992,1.0,0.9994,...,0.9985,0.9886,0.9996,0.9997,0.9997,0.9988,1.0,1.0,1.0,1.0
0,optdigits,5216,64,2.8758,0.7584,0.7631,0.7921,0.6934,0.7038,0.7412,...,0.7174,0.7732,0.6948,0.7126,0.7197,0.7483,0.6892,0.7659,0.7221,0.7606
0,pendigits,6870,16,2.2707,0.9443,0.9527,0.9213,0.9518,0.9352,0.9532,...,0.9229,0.9497,0.9349,0.9256,0.9442,0.9437,0.959,0.9222,0.9528,0.943


In [7]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,MCST,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP
0,arrhythmia,452,274,14.6018,0.5,0.5,0.5,0.5357,0.4643,0.5,...,0.6071,0.5714,0.5714,0.6071,0.5357,0.5357,0.5357,0.5,0.5714,0.5357
0,cardio,1831,21,9.6122,0.4429,0.5,0.5571,0.5143,0.5286,0.4857,...,0.4571,0.5,0.4714,0.5857,0.5,0.5286,0.4429,0.4857,0.4857,0.4
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.587,0.5652,0.5,0.587,0.587,0.5217,...,0.5217,0.5435,0.5217,0.5217,0.587,0.5,0.5217,0.5217,0.5435,0.5435
0,letter,1600,32,6.25,0.0488,0.0488,0.0976,0.0732,0.0976,0.0976,...,0.0732,0.0488,0.0732,0.0976,0.0488,0.0488,0.0488,0.0976,0.122,0.0976
0,lympho,148,18,4.0541,0.6667,1.0,0.6667,1.0,1.0,1.0,...,1.0,1.0,1.0,0.6667,1.0,1.0,0.6667,0.6667,0.6667,1.0
0,mnist,7603,100,9.2069,0.2407,0.2667,0.2926,0.2593,0.3296,0.263,...,0.2963,0.3185,0.3556,0.2556,0.3185,0.2852,0.2407,0.2815,0.3111,0.2704
0,musk,3062,166,3.1679,1.0,0.9024,0.9756,0.9512,1.0,0.9512,...,0.9024,0.7561,0.9024,0.9756,0.9512,0.9024,1.0,1.0,1.0,1.0
0,optdigits,5216,64,2.8758,0.0308,0.0308,0.0615,0.0,0.0308,0.0308,...,0.0154,0.0154,0.0154,0.0154,0.0462,0.0154,0.0154,0.0308,0.0154,0.0462
0,pendigits,6870,16,2.2707,0.2097,0.2903,0.2258,0.2581,0.2258,0.2903,...,0.2903,0.2742,0.2581,0.2097,0.2581,0.2419,0.3387,0.2419,0.2258,0.2742
